<a href="https://colab.research.google.com/github/ShkarupyloMaksym/lichess_datamining/blob/main/chess_data_mining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

For processing data in several notebooks

# Imports

In [ ]:
!pip install chess

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 1.7 MB/s eta 0:00:00


In [ ]:
import requests
import json
import os
import gdown
import re
import pandas as pd
import time
import numpy as np
import shutil

# Drive working

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
i = 1
checked_file_name = f'games_metadata_profile_{i}.csv'
drive_path = '/content/drive/MyDrive/path/to/your/data'

In [ ]:
def copy_file_from(path=drive_path, name=checked_file_name):
  shutil.copyfile(os.path.join(path, name), name)

def copy_file_to(path=drive_path, name=checked_file_name):
  shutil.copyfile(name, os.path.join(path, name))

In [ ]:
def chiter_function():
  print('But just kill me')
  copy_file_to()
  print("I'm like a phoenix")
  raise Exception('Killed')
  # restart_runtime()


In [ ]:
def convert_drive_link(original_link):
  if "https://drive.google.com/uc?id=" in original_link:
    return original_link

  pattern = r"https://drive\.google\.com/file/d/([a-zA-Z0-9_-]+)/view\?usp=drive_link"

  match = re.match(pattern, original_link)

  if match:
    file_id = match.group(1)
    converted_link = f"https://drive.google.com/uc?id={file_id}"
    return converted_link
  else:
    print("Invalid Google Drive link format.")
    return None

def install_from_google_drive(link, name, path=None, force_download = False):
  full_path = name
  if path is not None:
    full_path = os.path.join(path, full_path)
  if not force_download:
    if os.path.exists(full_path):
      print('The data already exists')
      return

  print('Start downloading')
  gdown.download(convert_drive_link(link), full_path, quiet=False)
  print('Downloading have ended')


In [ ]:
copy_file_from()

# install_from_google_drive('link_to_your_1_profile_file', 'games_metadata_profile_1.csv', force_download=False)
# install_from_google_drive('link_to_your_2_profile_file', 'games_metadata_profile_2.csv', force_download=False)
# install_from_google_drive('link_to_your_3_profile_file', 'games_metadata_profile_3.csv', force_download=False)
# install_from_google_drive('link_to_your_4_profile_file', 'games_metadata_profile_4.csv', force_download=False)
# install_from_google_drive('link_to_your_5_profile_file', 'games_metadata_profile_5.csv', force_download=False)
# install_from_google_drive('link_to_your_6_profile_file', 'games_metadata_profile_6.csv', force_download=False)
# install_from_google_drive('link_to_your_7_profile_file', 'games_metadata_profile_7.csv', force_download=False)
# install_from_google_drive('link_to_your_8_profile_file', 'games_metadata_profile_8.csv', force_download=False)
# install_from_google_drive('link_to_your_9_profile_file', 'games_metadata_profile_9.csv', force_download=False)
# install_from_google_drive('link_to_your_10_profile_file', 'games_metadata_profile_10.csv', force_download=False)
# install_from_google_drive('link_to_your_11_profile_file', 'games_metadata_profile_11.csv', force_download=False)
# install_from_google_drive('link_to_your_12_profile_file', 'games_metadata_profile_12.csv', force_download=False)

# API functions

In [ ]:
def get_profile(username, fields):
    profile = {}
    url = f'https://lichess.org/api/user/{username}'
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        account_info = response.json()
        for field in fields:
            try:
                keys = field.split('/')
                value = account_info
                for key in keys:
                    if key in value:
                        value = value.get(key, {})
                    else:
                        value = None
                        break
                profile[field] = value
            except Exception as e:
                print(f"Error processing {field} for {username}: {e}")
                profile[field] = None
    else:
      print(response.text)
      print('I want to sleep')

      chiter_function()

      time.sleep(60)
      print('I get sleep enough')
      profile = get_profile(username, fields)
    return profile

In [ ]:
def update_profiles(df, fields, name_for_saving):
  now = time.time()
  columns_names = ['White', 'Black']
  for column in columns_names:
      for field in fields:
          if f"{column}_{field.replace('/', '_')}" not in df.columns:
            df[f"{column}_{field.replace('/', '_')}"] = None

  started = True

  for index, row in df.iterrows():
      if not (pd.isnull(row['White_createdAt']) and pd.isnull(row['Black_createdAt'])):
        started = True
        continue
      if started:
        print(f'Started from {index}')
        started = False
      if index % 100 == 0 and index != 0:
        now_new = time.time()
        taken_time = round(now_new - now)
        taken_time_min, taken_time_sec = taken_time // 60, taken_time % 60
        taken_time_sec = str(taken_time_sec).zfill(2)
        print(f'{index} was completed, taken time = {taken_time_min}:{taken_time_sec}')
        df.to_csv(name_for_saving, index=False)

      for column in columns_names:
        username = row[column]
        profile = get_profile(username, fields)
        for field, value in profile.items():
            df.at[index, f"{column}_{field.replace('/', '_')}"] = value

In [ ]:
token = 'YOUR_LICHESS_TOKEN'

headers = {
    'Authorization': token
}

fields=["profile/flag", "createdAt", "playTime/total", "count/all", "tosViolation", "title"]

i=1

games_metadata = pd.read_csv(f'games_metadata_profile_{i}.csv')
update_profiles(games_metadata, fields, f'games_metadata_profile_{i}.csv')
# games_metadata.to_csv(f'games_metadata_profile_{i}.csv', index=False)
# print(f"{i} file was updated")